### N Gram In N Gram Analysis

In [3]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [4]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [5]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
twogram_start = 0  # 0  # native word start index
twogram_end = 200  # 28  # native word end index

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False

# n gram sample
threegram_sample = 6
fourgram_sample = 4
fivegram_sample = 2

# n gram select
threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

In [6]:
#path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Deployment2/Result/5-N Gram In N Gram Analysis/{lang_folder.capitalize()} {lang_pair.capitalize()}"
#
#Path(path).mkdir(parents=True, exist_ok=True)

In [7]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [8]:
def word_in_wordgroup(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [9]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
    df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]
    df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]

df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
195,beni dinle,40708
196,doğru mu,40701
197,bu o,40567
198,bir de,40481


In [10]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [11]:
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram = df_fourgram_sent.loc[:,["fourgram","frequency"]]
else:
    df_fourgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")  
    df_fourgram = df_fourgram.loc[:,["fourgram","frequency"]]

df_fourgram

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
3052209,fikret miloya merhaba de,5
3052210,fikret millsin oğlu mu,5
3052211,fikret millet iyi iş,5
3052212,fikret millet bize bakıyor,5


In [12]:
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram = df_fivegram_sent.loc[:,["fivegram","frequency"]]
else:
    df_fivegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")  
    df_fivegram = df_fivegram.loc[:,["fivegram","frequency"]]

df_fivegram

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [13]:
df_ngram_concat = pd.concat([df_twogram_select, df_threegram, df_fourgram, df_fivegram], axis=1)
df_ngram_concat

,twogram,frequency,threegram,frequency,fourgram,frequency,fivegram,frequency
0,bir şey,859944.0,bir şey yok,113165.0,bir şey var mı,41773,başka bir şey var mı,14104.0
1,değil mi,585879.0,bir şey var,110455.0,her şey yolunda mı,31126,bu da ne demek oluyor,10205.0
2,ben de,377765.0,bu da ne,89463.0,burada ne işin var,21993,o kadar da kötü değil,7012.0
3,teşekkür ederim,370619.0,her şey yolunda,75968.0,bir sorun mu var,21423,sence de öyle değil mi,6305.0
4,ne oldu,322758.0,başka bir şey,75193.0,ben de seni seviyorum,17338,sana bir şey sorabilir miyim,6224.0
...,...,...,...,...,...,...,...,...
3052209,NaN,NaN,NaN,NaN,fikret miloya merhaba de,5,NaN,NaN
3052210,NaN,NaN,NaN,NaN,fikret millsin oğlu mu,5,NaN,NaN
3052211,NaN,NaN,NaN,NaN,fikret millet iyi iş,5,NaN,NaN
3052212,NaN,NaN,NaN,NaN,fikret millet bize bakıyor,5,NaN,NaN


In [14]:
df_two_in_three = word_in_wordgroup(df_ngram_concat, "twogram","threegram",threegram_sample)
df_two_in_four = word_in_wordgroup(df_ngram_concat, "twogram","fourgram",fourgram_sample)
df_two_in_five = word_in_wordgroup(df_ngram_concat, "twogram","fivegram",fivegram_sample)

In [15]:
df_twogram_order_join_threegram = df_two_in_three.groupby(["twogram"])["threegram"].apply(", ".join).reset_index()
df_twogram_order_join_fourgram = df_two_in_four.groupby(["twogram"])["fourgram"].apply(", ".join).reset_index()
df_twogram_order_join_fivegram = df_two_in_five.groupby(["twogram"])["fivegram"].apply(", ".join).reset_index()  

In [16]:
dfs = [df_twogram_order_join_threegram, df_twogram_order_join_fourgram, df_twogram_order_join_fivegram]

In [17]:
df_ngram_sample_join_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='outer'), dfs)
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge 

,twogram,threegram,fourgram,fivegram
0,acele et,"hadi acele et, acele et ve, haydi acele et, lü...","acele et acele et, acele et o zaman, o zaman a...","acele et ve buraya gel, acele et nong fikret f..."
1,ama ben,"ama ben de, evet ama ben, ama ben bir, ama ben...","seni bilmem ama ben, sizi bilmem ama ben, ama ...","ama ben bir şey yapmadım, hayır ama ben seni t..."
2,ama bu,"evet ama bu, ama bu sefer, ama bu çok, ama bu ...","ama bu doğru değil, ama bu sadece bir, özür di...","ama bu uzun zaman önceydi, ama bu iyi bir şey"
3,aman tanrım,"oh aman tanrım, aman tanrım bu, aman tanrım ne...","aman tanrım aman tanrım, aman tanrım iyi misin...","aman tanrım bu da ne, aman tanrım çok özür dil..."
4,anladın mı,"beni anladın mı, istediğimi anladın mı, anladı...","demek istediğimi anladın mı, ne dediğimi anlad...","ne demek istediğimi anladın mı, ne demek isted..."
...,...,...,...,...
195,şey yolunda,"her şey yolunda, şey yolunda mı, şey yolunda g...","her şey yolunda mı, her şey yolunda gidecek, e...","orada her şey yolunda mı, burada her şey yolun..."
196,şimdi ne,"şimdi ne olacak, şimdi ne yapacağız, peki şimd...","peki şimdi ne olacak, peki şimdi ne yapacağız,...","ne şimdi ne de sonra, şimdi ne olacak biliyor ..."
197,şu an,"ama şu an, şu an için, şu an bir, şu an burada...","şu an burada değil, şu an biraz meşgulüm, şu a...","aradığınız kişiye şu an ulaşılamıyor, şu an iy..."
198,şu anda,"ama şu anda, şu anda bir, ve şu anda, şu anda ...","şu anda burada değil, şu anda nerede olduğunu,...","aradığınız kişiye şu anda ulaşılamıyor, aradığ..."


In [18]:
df_ngram_sample_join_merge = pd.merge(df_ngram_sample_join_merge, df_twogram_select, how="left", on="twogram")
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge.rename(columns={"frequency":"two_freq"}, inplace=True)
df_ngram_sample_join_merge.sort_values(by="two_freq", ascending=False, inplace=True)
df_ngram_sample_join_merge.reset_index(drop=True, inplace=True)
df_ngram_sample_join_merge

,twogram,threegram,fourgram,fivegram,two_freq
0,bir şey,"bir şey yok, bir şey var, başka bir şey, bir ş...","bir şey var mı, gereken bir şey var, başka bir...","başka bir şey var mı, sana bir şey sorabilir m...",859944
1,değil mi,"öyle değil mi, biliyorsun değil mi, güzel deği...","de öyle değil mi, bir şey değil mi, çok güzel ...","sence de öyle değil mi, sizce de öyle değil mi",585879
2,ben de,"ben de seni, ben de öyle, ve ben de, evet ben ...","ben de seni seviyorum, ben de öyle düşünmüştüm...","ben de seni gördüğüme sevindim, evet ben de öy...",377765
3,teşekkür ederim,"çok teşekkür ederim, için teşekkür ederim, teş...","için çok teşekkür ederim, geldiğiniz için teşe...","her şey için teşekkür ederim, zaman ayırdığını...",370619
4,ne oldu,"sonra ne oldu, ne oldu sana, sana ne oldu, ne ...","bil bakalım ne oldu, sana ne oldu böyle, ne ol...","az önce ne oldu öyle, sonra ne oldu biliyor musun",322758
...,...,...,...,...,...
195,beni dinle,"şimdi beni dinle, fikret beni dinle, sen beni ...","beni dinle tamam mı, hayır sen beni dinle, şim...","kapa çeneni ve beni dinle, kes sesini ve beni ...",40708
196,doğru mu,"bu doğru mu, doğru mu bu, olduğu doğru mu, doğ...","doğru mu yanlış mı, doğru mu değil mi, sence b...","bu doğru mu değil mi, bu doğru mu bay harding",40701
197,bu o,"bu o mu, bu o kadar, bu o değil, evet bu o, i̇...","bu o kadar da, bu o kadar kolay, bu o değil mi...","bu o kadar kolay değil, bu o kadar basit değil",40567
198,bir de,"ve bir de, bir de bana, bir de bu, bir de şu, ...","bir de bana sor, bir de bu var, bir de şu var,...","evet bir de bana sor, bir de iyi tarafından bak",40481


#### Adding Frequency For Ratio

In [19]:
df_two_in_three = pd.merge(df_two_in_three, df_twogram_select, how="left", on="twogram")
df_two_in_three.drop_duplicates(inplace=True)
df_two_in_three.rename(columns={"frequency":"two_freq"}, inplace=True)
df_two_in_three

,twogram,threegram,two_freq
0,bir şey,bir şey yok,859944
1,bir şey,bir şey var,859944
2,bir şey,başka bir şey,859944
3,bir şey,bir şey değil,859944
4,bir şey,bir şey mi,859944
...,...,...,...
1195,devam edin,lütfen devam edin,40036
1196,devam edin,aramaya devam edin,40036
1197,devam edin,yürümeye devam edin,40036
1198,devam edin,devam edin bay,40036


In [20]:
df_two_in_three_freq = pd.merge(df_two_in_three, df_threegram, how="left", on="threegram")
df_two_in_three_freq.drop_duplicates(inplace=True)
df_two_in_three_freq.rename(columns={"frequency":"three_freq"}, inplace=True)
df_two_in_three_freq = df_two_in_three_freq[["twogram","two_freq","threegram","three_freq"]]
df_two_in_three_freq

,twogram,two_freq,threegram,three_freq
0,bir şey,859944,bir şey yok,113165
1,bir şey,859944,bir şey var,110455
2,bir şey,859944,başka bir şey,75193
3,bir şey,859944,bir şey değil,68498
4,bir şey,859944,bir şey mi,64933
...,...,...,...,...
1195,devam edin,40036,lütfen devam edin,1567
1196,devam edin,40036,aramaya devam edin,1434
1197,devam edin,40036,yürümeye devam edin,1147
1198,devam edin,40036,devam edin bay,766


In [21]:
df_two_in_four_freq = pd.merge(df_two_in_four, df_fourgram, how="left", on="fourgram")
df_two_in_four_freq.drop_duplicates(inplace=True)
df_two_in_four_freq.rename(columns={"frequency":"four_freq"}, inplace=True)
df_two_in_four_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_four_freq

,fourgram,four_freq
0,bir şey var mı,41773
1,gereken bir şey var,15683
2,başka bir şey var,12619
3,bir şey daha var,12185
4,de öyle değil mi,5379
...,...,...
795,bir de ben deneyeyim,430
796,devam edin devam edin,388
797,devam edin bay fikret,184
798,bizi izlemeye devam edin,173


In [22]:
df_two_in_five_freq = pd.merge(df_two_in_five, df_fivegram, how="left", on="fivegram")
df_two_in_five_freq.drop_duplicates(inplace=True)
df_two_in_five_freq.rename(columns={"frequency":"five_freq"}, inplace=True)
df_two_in_five_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_five_freq

,fivegram,five_freq
0,başka bir şey var mı,14104
1,sana bir şey sorabilir miyim,6224
2,sence de öyle değil mi,6305
3,sizce de öyle değil mi,1930
4,ben de seni gördüğüme sevindim,1641
...,...,...
395,bu o kadar basit değil,766
396,evet bir de bana sor,481
397,bir de iyi tarafından bak,279
398,zik zak yapmaya devam edin,84


In [23]:
df_ngram_sample_concat = pd.concat([df_two_in_three_freq, df_two_in_four_freq, df_two_in_five_freq], axis=1)
df_ngram_sample_concat

,twogram,two_freq,threegram,three_freq,fourgram,four_freq,fivegram,five_freq
0,bir şey,859944,bir şey yok,113165,bir şey var mı,41773.0,başka bir şey var mı,14104.0
1,bir şey,859944,bir şey var,110455,gereken bir şey var,15683.0,sana bir şey sorabilir miyim,6224.0
2,bir şey,859944,başka bir şey,75193,başka bir şey var,12619.0,sence de öyle değil mi,6305.0
3,bir şey,859944,bir şey değil,68498,bir şey daha var,12185.0,sizce de öyle değil mi,1930.0
4,bir şey,859944,bir şey mi,64933,de öyle değil mi,5379.0,ben de seni gördüğüme sevindim,1641.0
...,...,...,...,...,...,...,...,...
1195,devam edin,40036,lütfen devam edin,1567,NaN,NaN,NaN,NaN
1196,devam edin,40036,aramaya devam edin,1434,NaN,NaN,NaN,NaN
1197,devam edin,40036,yürümeye devam edin,1147,NaN,NaN,NaN,NaN
1198,devam edin,40036,devam edin bay,766,NaN,NaN,NaN,NaN


In [24]:
twogram_select_freq = df_twogram_select["frequency"].sum()
threegram_result_freq = df_ngram_sample_concat["three_freq"].sum()
fourgram_result_freq = df_ngram_sample_concat["four_freq"].sum()
fivegram_result_freq = df_ngram_sample_concat["five_freq"].sum()

In [26]:
df_threegram.iloc[threegram_select_start:threegram_select_end,]["frequency"].sum()

21486996

In [69]:
df_ngram_sample_concat["five_freq"].sum()

428522.0